In [1]:
import os

In [2]:
# Confirm the current working directory
%pwd

'D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00015_MLOps_End-to-End-ML-Application-Project_MLflow\\research'

In [3]:
# Change the directory to one level below the current working directory
os.chdir('../')

In [4]:
# Confirm the change of directory
%pwd

'D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00015_MLOps_End-to-End-ML-Application-Project_MLflow'

In [5]:
# Creating the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# Import the needed contants and methods
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
# Define the configuration manager
class ConfigurationManager:
    
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH, 
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir=config.root_dir,
        source_URL=config.source_URL,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
    )
        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
# Update components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config  
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f'{filename} download! with the following info: \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.Zipfile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Update pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-01 18:42:46,286: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-01 18:42:46,309: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-01 18:42:46,323: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 18:42:46,353: INFO: common: created directory at: artifacts]
[2023-10-01 18:42:46,423: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-01 18:42:47,257: INFO: 2494227342: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 21952
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4fb73de802f6b222673e69a029c67977b6b2e425860a2876156f364c6c16de21"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AB2A:7590:147F3A9:1570EA6:6519AF96
Accept-Ranges: bytes
Date: Sun

AttributeError: module 'zipfile' has no attribute 'Zipfile'